# Assignment 1 - Building a Random Forest

This is a skeleton of a Random Forest classifier for the example data set in `data/example.csv`.

In [64]:
import csv
import math
from statistics import median, mode, mean
from collections import Counter
from enum import Enum
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing

Some simple type definitions.

In [65]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label


class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2


class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split

Also, some basic classes to represent an attribute, a spltting procedure, and a node.

In [66]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes


class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition


class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

The input filename is hard-coded.

In [67]:
INFILE = "data/housing_price_train.csv"
test = "data/housing_price_test.csv"

The attribute labels types are hard-coded too (the same order as in the file!).

In [68]:
ATTR = [Attribute("0", AttrType.num), Attribute("1", AttrType.num),
        Attribute("2", AttrType.num), 
        Attribute("3", AttrType.num), Attribute("4", AttrType.num),
        Attribute("target", AttrType.target)]

The index of the target attribute (assuming it's the last).

In [69]:
IDX_TARGET = len(ATTR) - 1 

In [70]:
data = pd.read_csv(INFILE)
test_data = pd.read_csv(test, index_col ="Id")

In [71]:
test_index = test_data.index

In [72]:
data.replace(to_replace=["?", "none", "None"], value=np.nan, inplace=True)
data.dropna(thresh=1400, inplace=True, axis=1)
c = data.columns
test_data1 = test_data.filter(c)

null_columns=test_data1.columns[test_data1.isna().any()]
test_data1[null_columns].isna().sum()
null_columns2=data.columns[data.isnull().any()]
data[null_columns2].isnull().sum()

MasVnrArea       8
BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
dtype: int64

In [73]:
data["MasVnrArea"].replace(to_replace=np.nan, value=0.0, inplace=True)
data["BsmtQual"].replace(to_replace=np.nan, value="TA", inplace=True)
data["BsmtCond"].replace(to_replace=np.nan, value="TA", inplace=True)
data["BsmtExposure"].replace(to_replace=np.nan, value="No", inplace=True)
data["BsmtFinType1"].replace(to_replace=np.nan, value="Unf", inplace=True)
data["BsmtFinType2"].replace(to_replace=np.nan, value="Unf", inplace=True)
data["Electrical"].replace(to_replace=np.nan, value="SBrkr", inplace=True)

In [74]:
test_data1["MSZoning"].replace(to_replace=np.nan, value="RL", inplace=True)
test_data1["MasVnrArea"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["Utilities"].replace(to_replace=np.nan, value="AllPub", inplace=True)
test_data1["Exterior1st"].replace(to_replace=np.nan, value="VinylSd", inplace=True)
test_data1["Exterior2nd"].replace(to_replace=np.nan, value="VinylSd", inplace=True)
test_data1["BsmtQual"].replace(to_replace=np.nan, value="TA", inplace=True)
test_data1["BsmtCond"].replace(to_replace=np.nan, value="TA", inplace=True)
test_data1["BsmtExposure"].replace(to_replace=np.nan, value="No", inplace=True)
test_data1["BsmtFinType1"].replace(to_replace=np.nan, value="GLQ", inplace=True)
test_data1["BsmtFinSF1"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtFinType2"].replace(to_replace=np.nan, value="Unf", inplace=True)
test_data1["BsmtFinSF2"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtUnfSF"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["TotalBsmtSF"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtFullBath"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtHalfBath"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["KitchenQual"].replace(to_replace=np.nan, value="TA", inplace=True)
test_data1["Functional"].replace(to_replace=np.nan, value="Typ", inplace=True)
test_data1["GarageCars"].replace(to_replace=np.nan, value=2.0, inplace=True)
test_data1["GarageArea"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["SaleType"].replace(to_replace=np.nan, value="WD", inplace=True)

In [75]:
train_y = data.iloc[:,-1]
train_x = data.drop(["SalePrice", "Id"], axis=1)

In [76]:
scaler = preprocessing.StandardScaler()
scaler.fit(pd.get_dummies(train_x))
train_x_s = pd.DataFrame(scaler.transform(pd.get_dummies(train_x)))

In [77]:
train_x_s

,0,1,2,3,4,5,6,7,8,9,...,235,236,237,238,239,240,241,242,243,244
0,0.073375,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,-0.288653,-0.944591,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1,-0.872563,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,-0.288653,-0.641228,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
2,0.073375,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,-0.288653,-0.301643,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
3,0.309859,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,-0.288653,-0.061670,...,-0.058621,-0.301962,-0.045376,0.390293,3.668167,-0.052414,-0.091035,-0.117851,-2.138345,-0.305995
4,0.073375,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,-0.288653,-0.174865,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.073375,-0.260560,-0.071836,-0.517200,0.918511,0.733308,-0.570750,-0.973018,-0.288653,0.873321,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1456,-0.872563,0.266407,-0.071836,0.381743,0.222975,0.151865,0.087911,0.759659,0.722112,0.049262,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1457,0.309859,-0.147810,0.651479,3.078570,-1.002492,1.024029,-0.570750,-0.369871,-0.288653,0.701265,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1458,-0.872563,-0.080160,-0.795151,0.381743,-0.704406,0.539493,-0.570750,-0.865548,6.092188,-1.284176,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995


In [78]:
scaler2 = preprocessing.StandardScaler()
scaler2.fit(pd.get_dummies(test_data1))
test_s = pd.DataFrame(scaler2.transform(pd.get_dummies(test_data1)))

In [79]:
test_s

,0,1,2,3,4,5,6,7,8,9,...,224,225,226,227,228,229,230,231,232,233
0,-0.869741,0.233137,-0.762849,0.396139,-0.340832,-1.075678,-0.565921,0.058428,0.523772,-0.650186,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
1,-0.869741,0.603300,-0.062823,0.396139,-0.439679,-1.217674,0.043095,1.059260,-0.298227,-0.337817,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
2,0.065900,0.542142,-0.762849,-0.504701,0.845332,0.675602,-0.565921,0.768909,-0.298227,-0.955663,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
3,0.065900,0.003062,-0.062823,0.396139,0.878281,0.675602,-0.453140,0.353179,-0.298227,-0.526157,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
4,1.469360,-0.692900,1.337229,-0.504701,0.680587,0.391611,-0.565921,-0.392496,-0.298227,1.065543,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1.469360,-0.946066,0.637203,-0.504701,1.043026,0.959594,-0.464418,-0.970998,-0.298227,1.885509,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
1556,-0.869741,-0.359404,-0.762849,0.396139,-0.406730,0.864930,-0.565921,-0.080148,-0.298227,0.240983,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511
1557,2.405001,-0.948305,0.637203,-0.504701,1.108924,1.006926,-0.565921,-0.970998,-0.298227,0.107767,...,-0.043895,-0.2977,-0.050702,0.399814,3.862430,-0.071796,-0.09167,-0.137629,-2.147443,-0.301511
1558,-0.869741,0.914684,-0.062823,0.396139,-0.176087,0.864930,-0.565921,-0.427690,3.771810,0.009003,...,-0.043895,-0.2977,-0.050702,0.399814,-0.258904,-0.071796,-0.09167,-0.137629,0.465670,-0.301511


In [80]:
pca = PCA(n_components=5)
pca_df = pd.DataFrame(pca.fit_transform(pd.get_dummies(train_x_s)))

pca_test = pd.DataFrame(pca.fit_transform(pd.get_dummies(test_s)))
pca_df.insert(5, "target", train_y)

A class DT representing the decision tree classifier. It could represent with methods:

  - a given impurity measure;
  - the search for the best attribute to split with;
  - the addition of a node to the tree;
  - a convenient model printer;
  - the recursive call for obtaining a tree;
  - a builder and an applier.

In [81]:
class DT(object):
    def __init__(self, dataframe):
        self.data = dataframe.reset_index(drop=True)  # training data set (loaded into memory)
        self.model = []  # decision tree model
        self.default_target = 0.0  # default target class


    def __find_best_attr(self, attrs, records):
        """
        Finds the attribute with the largest gain.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :return:
        """
        splittings = []  # holds the splitting information for each attribute

        for a in attrs:
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split
            # splitting condition depends on the attribute type
            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                split_mode = SplitType.bin
                temp = self.data.loc[self.data.index.isin(records)]
                split_cond = np.mean(temp[a])  # (i.e., if less or equal than this value)
                list1 = temp.loc[temp[a] >= split_cond].index
                list2 = temp.loc[temp[a] < split_cond].index
                s = {
                    "right":list(list1),
                    "left": list(list2)
                }
                # TODO collect training records for each split (in `splits`)
                splits[a] = s
               
            # TODO compute gain for attribute a
            tempLeft = self.data.loc[self.data.index.isin(splits[a]["left"])]
            tempRight = self.data.loc[self.data.index.isin(splits[a]["right"])]

            msee = len(records) * np.var(self.data["target"].values) - (len(tempLeft) * np.var(tempLeft["target"].values) + len(tempRight) * np.var(tempRight["target"].values))
            splitting = Splitting(a, msee, split_mode, split_cond, splits[a])
            splittings.append(splitting)
        
        # find best splitting
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting

    def __add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,
                   split_cond=None):
        """
        Adds a node to the decision tree.

        :param parent_id:
        :param node_type:
        :param edge_value:
        :param val:
        :param split_type:
        :param split_cond:
        :return:
        """
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # also add it as a child of the parent node
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)

        return node_id

    def __id3(self, attrs, records, parent_id=None, value=None):
        """
        Function ID3 that returns a decision tree.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :param parent_id: ID of parent node
        :param value: Value corresponding to the parent attribute, i.e., label of the edge on which we arrived to this node
        :return:
        """
        # empty training set or empty set of attributes => create leaf node with default class
        if len(records) < 10 or not attrs:
            temp = self.data.loc[self.data.index.isin(records)]
            val = np.mean(temp["target"].values)
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=val)
            return

        # find the attribute with the largest gain
        splitting = self.__find_best_attr(attrs, records)
        # add node
        node_id = self.__add_node(parent_id, edge_value=value, val=splitting.attr, split_type=splitting.split_type,
                                  split_cond=splitting.cond)
        # TODO call tree construction recursively for each split
        self.__id3(attrs, list(splitting.splits["right"]), node_id, "right")
        self.__id3(attrs, list(splitting.splits["left"]), node_id, "left")

    def print_model(self, node_id=0, level=0):
        node = self.model[node_id]
        indent = "  " * level
        if node.type == NodeType.leaf:
            print(indent + str(node.edge_value) + " [Leaf node] class=" + str(node.val))
        else:
            cond = " <= " + str(node.split_cond) if ATTR[node.val].type == AttrType.num else " == ? "
            if node.type == NodeType.root:
                print("[Root node] '" + ATTR[node.val].label + "'" + cond)
            else:
                print(indent + str(node.edge_value) + " [Internal node] '" + ATTR[node.val].label + "'" + cond)
            # print tree for child notes recursively
            for n_id in node.children:
                self.print_model(n_id, level + 1)

    def build_model(self):
        self.model = []  # holds the decision tree model, represented as a list of nodes
        # Get majority class
        #   Note: Counter returns a dictionary, most_common(x) returns a list with the x most common elements as
        #         (key, count) tuples; we need to take the first element of the list and the first element of the tuple
        self.default_target = mean(self.data.target)
        self.__id3(set(range(len(ATTR) - 1)), list(range(len(self.data))))

    def __apply_model(self, record):
        node = self.model[0]
        while node.type != NodeType.leaf:
        	# TODO based on the value of the record's attribute that is tested in `node`,
        	# set `node` to one of its child nodes until a leaf node is reached
            child = None
            if (record[node.val] > float(node.split_cond)):
                child = "right"
            else:
                child = "left"
            for nchild in node.children:
                if self.model[nchild].edge_value == child:
                    node = self.model[nchild]
        return node.val
    
    def predict(self, records):
        predictions = []
        pred_val = self.__apply_model(records)
        # TODO append pred_val to predictions
        predictions.append(pred_val)
        return np.mean(predictions)

A class RF representing the random forest classifier. It could represent with methods:

  - subsampling from the dataset
  - build a model with multiple decision trees
  - apply the model

In [82]:
class RF(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.trees = []  # decision trees
        
    def __subsampling(self, train_set, sample_size_ratio):
        sample_number = round(len(train_set) * sample_size_ratio)
        # TODO: generate a subsample with replacement
        return train_set.sample(sample_number, replace=True)
        
    def build_model(self, train_set, number_of_trees):
        for i in range(number_of_trees):
            sample = self.__subsampling(train_set, 0.5)
            tree = DT(sample) # build a tree with sample data and split conditions
            tree.build_model()
            self.trees.append(tree)
            
    def predict(self, test_set):
        rf_predictions = []
        for row in test_set:
            predictions = [tree.predict(row) for tree in self.trees]
            rf_predictions.append(np.mean(predictions))
        return rf_predictions

In [83]:
def createSubmission(test_ids, predictions):
    sub = pd.DataFrame()
    sub['Id'] = test_ids
    sub['SalePrice'] = predictions
    sub.to_csv('submission.csv',index=False)

Finally, the main function building a random forest model, printing it and applying it on some unseen records.

In [84]:
def main():

    rf = RF()
    print("Build model:")
    rf.build_model(pca_df,5)
    
    pred = rf.predict(pca_test.values)
    createSubmission(test_index, pred)

if __name__ == "__main__":
    main()

Build model:
